<a href="https://colab.research.google.com/github/solver-Mart1n/data-science/blob/solver-Mart1n-c06w1p2-basic-sql/3_Lab_INSERT_UPDATE_DELETE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> COUNT, DISTINCT, LIMIT in SQL using Jupyter Notebooks </h1>

## Description
In this lab, you will learn some commonly used DML (Data Manipulation Language) statements of SQL other than SELECT. First, you will learn the
INSERT statement, which is used to insert new rows into a table. Next, you will learn the UPDATE statement which is used to update the data in
existing rows in the table. Lastly, you will learn the DELETE statement which is used to remove rows from a table.

<h3>Objectives</h3>
<h4>After completing this lab, you will be able to:</h4>
    <ul>
        <li>Insert new rows into a table</li>
        <li>Update data in existing rows of the table</li>
        <li>Remove rows from a table</li>
    </ul>


<h3>Table of Contents</h3>
    <ul>
        <li>Building the Database from an Internet Source</li>
        <li>Exploring the Database</li>
        <li>Using INSERT Statement</li>
        <li>Using UPDATE Statement</li>
        <li>Using DELETE Statement</li>
        <li>Practice Exercises INSERT, UPDATE, DELETE</li>
    </ul>

<p>Estimated Time Needed: <strong>30 min</strong></p>
<hr>

## Building the Database from an Internet Source
The database used in this lab comes from the following dataset source: [Film Locations in San Francisco](https://data.sfgov.org/Culture-and-Recreation/Film-Locations-in-San-Francisco/yitu-d5am/about_data) under a [PDDL: Public Domain Dedication and License](http://opendatacommons.org/licenses/pddl/1.0/).

### Ingesting a CSV from a Data Source Endpoint
Three API parameters are used on top of the base URL for the data source. This data paging is in compliance with the row limit of the endpoint API.

In [1]:
url1 = 'https://data.sfgov.org/resource/yitu-d5am.csv?$limit=1000&$offset=0'
url2 = 'https://data.sfgov.org/resource/yitu-d5am.csv?$limit=1000&$offset=999'
url3 = 'https://data.sfgov.org/resource/yitu-d5am.csv?$limit=49&$offset=1999'

The three will be ingested as data frames and combined into one. And this will be used to generate a CSV.

In [ ]:
!pip install pandas==1.3.3

In [3]:
import pandas as pd

In [ ]:
# Read the dataset from a csv file
df1 = pd.read_csv(url1, header=0, sep=",")

# Display the first few rows of the DataFrame
df1.tail()

In [ ]:
# Read the dataset from a csv file
df2 = pd.read_csv(url2, header=0, sep=",")

# Display the first few rows of the DataFrame
df2.head()

In [ ]:
df2.tail()

In [ ]:
# Read the dataset from a csv file
df3 = pd.read_csv(url3, header=0, sep=",")

# Display the first few rows of the DataFrame
df3.head()

In [ ]:
df3.tail()

Visible from the tail() and head() print outs of the three data frames, indices start from 0 each time a CSV is loaded. The "ignore_index" attribute of the data frame append() function is set to _True_ in order to avoid copying repeating indices to the aggregated data frame.

In [ ]:
df = df1.append(df2,ignore_index=True)
df = df.append(df3,ignore_index=True)
len(df)

In [ ]:
df.columns

In [11]:
df.drop(columns=[':@computed_region_6qbp_sg9q', ':@computed_region_ajp5_b2md', ':@computed_region_26cr_cadq'],inplace=True)

In [12]:
df.set_axis(['Title', 'ReleaseYear', 'Locations', 'FunFacts', 'ProductionCompany', 'Distributor', 'Director', 'Writer', 'Actor1', 'Actor2', 'Actor3'], axis='columns', inplace=True)

As a result, the index labels of the combined data frame starts from 0 and ends with 2048. This indicates a successful merge of the pages of data from the [Data SF](https://data.sfgov.org/resource/yitu-d5am.csv) source URL.

In [ ]:
df

Store the combined data frame into one CSV file.

In [14]:
df.to_csv('san_francisco_film_locations.csv', index=False)

### Create an SQL Database from the Pandas Data Frame

#### Option 1: Using Duck DB
You can create a duck database using a CSV file and the **CREATE OR REPLACE TABLE** and **AS FROM** directives with the _read_csv_auto()_ method.

In [ ]:
%pip install jupysql --upgrade duckdb-engine --quiet

In [ ]:
%reload_ext sql

In [ ]:
%sql duckdb:///san_francisco_film_locations.duck.db

In [ ]:
%%sql
CREATE OR REPLACE TABLE san_francisco_film_locations AS
FROM read_csv_auto('san_francisco_film_locations.csv', header=True, sep=',')

Proceed to the section: Exploring the Database

#### Option 2: Using SQL Alchemy

In [15]:
import sqlite3 as sq3

In [16]:
conn = sq3.connect('san_francisco_film_locations.db')
#df.to_sql('san_francisco_film_locations', conn, if_exists='append', index=False)
df.to_sql('san_francisco_film_locations', conn, if_exists='replace', index=False)

In [17]:
!pip install sqlalchemy

In [18]:
%reload_ext sql

In [19]:
%sql sqlite:///san_francisco_film_locations.db

## Exploring the Database
Now that we have a database. We can start exploring it through the _SELECT_ command. _FROM_ specifies the database to query. And the '*' specifies all its contents.

A value of 5 passed to the _LIMIT_ command limits the print out to only 5 records.

In [20]:
%%sql
SELECT *
FROM san_francisco_film_locations
LIMIT 5

 * sqlite:///san_francisco_film_locations.db
Done.


Title,ReleaseYear,Locations,FunFacts,ProductionCompany,Distributor,Director,Writer,Actor1,Actor2,Actor3
Experiment in Terror,1962,The Sea Captain's Chest (Fisherman's Wharf),None,Columbia Pictures Corporation,Columbia Pictures,Blake Edwards,The Gordons,Glenn Ford,Lee Remick,Stefanie Powers
Experiment in Terror,1962,100 St. Germain Avenue,None,Columbia Pictures Corporation,Columbia Pictures,Blake Edwards,The Gordons,Glenn Ford,Lee Remick,Stefanie Powers
Chan is Missing,1982,"Li Po (916 Grant Avenue at Washington, Chinatown)",None,New Yorker Films,New Yorker Films,Wayne Wang,Wayne Wang,Wood Moy,Marc Hayashi,Lauren Chew
A View to a Kill,1985,Taylor and Jefferson Streets (Fisherman's Wharf),None,Metro-Goldwyn Mayer,MGM/UA Entertainment Company,John Glen,Richard Maibaum,Roger Moore,Christopher Walken,Tanya Roberts
The Californians,2005,None,None,Parker Film Company,Fabrication Films,Jonathan Parker,Jonathan Parker & Catherine DiNapoli,Noah Wyle,None,None


These are the column attribute descriptions from the **san_francisco_film_locations** table:

|Column|Description|
|---|---|
|   Title| titles of the films|
|   ReleaseYear| time of public release of the films|
|   Locations| locations of San Francisco where the films were shot|
|   FunFacts| funny facts about the filming locations|
|   ProductionCompany| companies who produced the films|
|   Distributor| companies who distributed the films|
|   Director| people who directed the films|
|   Writer| people who wrote the films|
|   Actor1| person 1 who acted in the films|
|   Actor2| person 2 who acted in the films|
|   Actor3| person 3 who acted in the films|






## Using INSERT Statement
In this exercise, you will first go through some examples of using INSERT in queries and then solve some exercise problems by using it.


### Example 1
In this example, suppose we want to insert a new single row into the Instructor table.

1. Problem:
Insert a new location record with:
* the title **Some Fictitious Movie**, released on **2024**, filmed at **Some Street in San Francisco**, no fun facts, by **Some Production Inc.** production company through **Some Distributor**, directed by **Mr. Director**, written by **Mr. Writer**, with three lead actors **Mrs. Actor**, **Mr. Actor**, and **Ms. Actor**, into the san_francisco_film_locations table.

2. Solution:
The following inserts the new record to the Instructor database.

In [ ]:
%%sql
INSERT INTO san_francisco_film_locations(Title, ReleaseYear, Locations, FunFacts, ProductionCompany, Distributor, Director, Writer, Actor1, Actor2, Actor3)
VALUES('Some Fictitious Movie', 2024, 'Some Street in San Francisco', 'None', 'Some Production Inc.', 'Some Distributor', 'Mr. Director', 'Mr. Writer', 'Mrs. Actor', 'Mr. Actor', 'Ms. Actor');

3. Check if the new entry was included.

In [ ]:
%%sql
SELECT * FROM san_francisco_film_locations;

### Example 2
In this example, suppose we want to insert some new multiple rows into the Instructor table.
1. Problem:
Insert two new location records into the san_francisco_film_locations table.
* First record with the title **Another Fictitious Movie**, released on **2025**, filmed at **Next Street in San Francisco**, no fun facts, by **Some Production Inc.** production company through **Some Distributor**, directed by **Mr. Director A**, written by **Mr. Writer B**, with three lead actors **Mr. Actor C**, **Ms. Actor D**, and **Mr. Actor E**.
* Second record with the title **One Fictitious Movie**, released on **2026**, filmed at **That Street in San Francisco**, no fun facts, by **Some Production Inc.** production company through **Some Distributor**, directed by **Mr. Director N**, written by **Mr. Writer N+1**, with three lead actors **Mrs. Actor N+2**, **Mr. Actor N+3**, and **Ms. Actor N+4**.
2. Solution:

In [ ]:
%%sql
INSERT INTO san_francisco_film_locations(Title, ReleaseYear, Locations, FunFacts, ProductionCompany, Distributor, Director, Writer, Actor1, Actor2, Actor3)
VALUES('Another Fictitious Movie', 2025, 'Next Street in San Francisco', 'None', 'Some Production Inc.', 'Some Distributor', 'Mr. Director A', 'Mr. Writer B', 'Mrs. Actor C', 'Mr. Actor D', 'Ms. Actor E'),('One Fictitious Movie', 2026, 'That Street in San Francisco', 'None', 'Some Production Inc.', 'Some Distributor', 'Mr. Director N', 'Mr. Writer N+1', 'Mrs. Actor N+2', 'Mr. Actor N+3', 'Ms. Actor N+4');

3. Check if the new entry was included.

In [ ]:
%%sql
SELECT *
FROM san_francisco_film_locations

##Using UPDATE Statement
Let us go through some examples of UPDATE related queries:

### Example 3
In this example, we want to update one column of an existing row of the table.
1. Problem:
Update the city for Sandip to Toronto.
2. Solution:
UPDATE Instructor
SET city='Toronto'
WHERE firstname="Sandip";
3. Copy the solution code above by clicking on the little copy button on the bottom right of the codeblock below and paste it to the textbox
of the Datasette tool. Then click Submit query.
4. Copy the code below by clicking on the little copy button on the bottom right of the codeblock below and paste it to the textbox of the
Datasette tool. Then click Submit query.
SELECT * FROM Instructor;

### Example 4
In this example, we want to update multiple columns of an existing row of the table.
1. Problem:
Update the city and country for Doe with id 5 to Dubai and AE respectively.
2. Solution:
UPDATE Instructor
SET city='Dubai', country='AE'
WHERE ins_id=5;
3. Copy the solution code above by clicking on the little copy button on the bottom right of the codeblock below and paste it to the textbox
of the Datasette tool. Then click Submit query.
4. Copy the code below by clicking on the little copy button on the bottom right of the codeblock below and paste it to the textbox of the
Datasette tool. Then click Submit query.
SELECT * FROM Instructor;

## Using DELETE Statement
In this exercise, you will first go through an example of using DELETE in a query and then solve an exercise problem by using it.

Let us go through an example of a DELETE related query:

### Example 5
In this example, we want to remove a row from the table.
1. Problem:
Remove the instructor record of Doe whose id is 6.
2. Solution:
DELETE FROM instructor
WHERE ins_id = 6;
3. Copy the solution code above by clicking on the little copy button on the bottom right of the codeblock below and paste it to the textbox
of Custom SQL query of the Datasette tool. Then click Submit query.
4. Copy the code below by clicking on the little copy button on the bottom right of the codeblock below and paste it to the textbox of the
Datasette tool. Then click Submit query.
SELECT * FROM Instructor;

## Practice Exercises

### INSERT
Now, let us practice creating and running some INSERT related queries.

#### Problem 1
Insert a new instructor record with First record with the title **Movie 001**, released on
**2027**, filmed at **1 Street in San Francisco**, no fun facts, by **Some Production Inc.** production company through **Some Distributor**, directed by **Director 1**, written by **Writer 1**, with three lead actors **Actor 10**, **Actor 20**, and **Actor 30** into the san_francisco_film_locations table.

<details><summary>Hint</summary>


```
Follow example 1 of the INSERT exercise.
%%sql
INSERT INTO tablename([column1], [column2], [column3], [column4], [column5], [column6], [column7], [column8], [column9], [column10], [column11])
VALUES('Movie 001', 2027, '1 Street in San Francisco', 'None', 'Some Production Inc.', 'Some Distributor', 'Director 1', 'Writer 1', 'Actor 10', 'Actor 20', 'Actor 30');

SELECT * FROM tablename;
```

</details>

<details><summary>Query Solution</summary>

```
%%sql
INSERT INTO san_francisco_film_locations(Title, ReleaseYear, Locations, FunFacts, ProductionCompany, Distributor, Director, Writer, Actor1, Actor2, Actor3)
VALUES('Movie 001', 2027, '1 Street in San Francisco', 'None', 'Some Production Inc.', 'Some Distributor', 'Director 1', 'Writer 1', 'Actor 10', 'Actor 20', 'Actor 30');

SELECT * FROM san_francisco_film_locations
```

</details>

In [ ]:
%%sql
INSERT INTO san_francisco_film_locations(Title, ReleaseYear, Locations, FunFacts, ProductionCompany, Distributor, Director, Writer, Actor1, Actor2, Actor3)
VALUES('Movie 001', 2027, '1 Street in San Francisco', 'None', 'Some Production Inc.', 'Some Distributor', 'Director 1', 'Writer 1', 'Actor 10', 'Actor 20', 'Actor 30');

SELECT * FROM san_francisco_film_locations

#### Problem 2
Insert two new instructor records into the “Instructor” table.
* First record is a sequel of Another Fictitious Movie.
* Second
record is a sequel of One Fictitious Movie.

<details><summary>Hint</summary>

```
Follow example 2 of the INSERT exercise.
```

</details>

<details><summary>Query Solution</summary>

```
%%sql
INSERT INTO san_francisco_film_locations(Title, ReleaseYear, Locations, FunFacts, ProductionCompany, Distributor, Director, Writer, Actor1, Actor2, Actor3)
VALUES('Another Fictitious Movie Part 2', 2026, 'Next Street in San Francisco', 'None', 'Some Production Inc.', 'Some Distributor', 'Mr. Director A', 'Mr. Writer B', 'Mrs. Actor C', 'Mr. Actor D', 'Ms. Actor E'),('One Fictitious Movie Part 2', 2027, 'That Street in San Francisco', 'None', 'Some Production Inc.', 'Some Distributor', 'Mr. Director N', 'Mr. Writer N+1', 'Mrs. Actor N+2', 'Mr. Actor N+3', 'Ms. Actor N+4');

SELECT * from san_francisco_film_locations;
```

</details>

In [ ]:
%%sql
INSERT INTO san_francisco_film_locations(Title, ReleaseYear, Locations, FunFacts, ProductionCompany, Distributor, Director, Writer, Actor1, Actor2, Actor3)
VALUES('Another Fictitious Movie Part 2', 2026, 'Next Street in San Francisco', 'None', 'Some Production Inc.', 'Some Distributor', 'Mr. Director A', 'Mr. Writer B', 'Mrs. Actor C', 'Mr. Actor D', 'Ms. Actor E'),('One Fictitious Movie Part 2', 2027, 'That Street in San Francisco', 'None', 'Some Production Inc.', 'Some Distributor', 'Mr. Director N', 'Mr. Writer N+1', 'Mrs. Actor N+2', 'Mr. Actor N+3', 'Ms. Actor N+4');

SELECT * from san_francisco_film_locations;

### UPDATE
Now, let us practice creating and running some UPDATE related queries.

#### Problem 3
Update the city of the instructor record to Markham whose id is 1.

<details><summary>Hint</summary>

```
Follow example 1 of the UPDATE exercise.
%%sql
UPDATE table_name
SET [column1]='Markham'
WHERE [specifiedcolumn]=1;
SELECT * FROM tablename;
```

</details>

<details><summary>Query Solution</summary>

```
%%sql
UPDATE Instructor
SET city='Markham'
WHERE ins_id=1;
SELECT * FROM Instructor;

```

</details>

#### Problem 4
Update the city and country for Sandip with id 4 to Dhaka and BD respectively.

<details><summary>Hint</summary>

```
Follow example 2 of the UPDATE exercise.
UPDATE table_name
SET [column1]='Dhaka', [column2]='BD'
WHERE [specifiedcolumn]=4;
SELECT * FROM tablename;
```

</details>

<details><summary>Query Solution</summary>

```
%%sql
UPDATE Instructor
SET city='Dhaka', country='BD'
WHERE ins_id=4;
SELECT * FROM Instructor;
```

</details>

### DELETE
Now, let us practice creating and running a DELETE related query.

#### Problem 5
Remove the instructor record of Hima.

<details><summary>Hint</summary>

```
Follow example 1 of the DELETE exercise.
DELETE FROM tablename
WHERE [columnanme1] = 'Hima';
SELECT * FROM tablename;
```

</details>

<details><summary>Query Solution</summary>

```
%%sql
DELETE FROM instructor
WHERE firstname = 'Hima';
SELECT * FROM Instructor;
```

</details>

2. Retrieve the first 10 film names released in 2015.

<details><summary>Hint</summary>

```
Follow example 1 of LIMIT. Use DISTINCT. Use WHERE clause comparison operator =, which means Equal to.
```

</details>

<details><summary>Query Solution</summary>

```
%%sql
SELECT DISTINCT Title FROM san_francisco_film_locations WHERE ReleaseYear=2015 LIMIT 10
```

</details>

3. Retrieve the next 3 film names that follow after the first 5 films released in 2015.

<details><summary>Hint</summary>

```
Follow example 2 of the LIMIT exercise to learn how to use OFFSET. Use DISTINCT and use the WHERE clause comparison operator =, which
means Equal to.
```

</details>

<details><summary>Query Solution</summary>

```
%%sql
SELECT DISTINCT Title FROM san_francisco_film_locations WHERE ReleaseYear=2015 LIMIT 3 OFFSET 5
```

</details>

## Conclusion
Thank you for completing the lab! You are now able to perform operations on tables like:

*   Inserting rows using the INSERT command
*   Removing rows using DELETE
*   Updating the data in existing rows with UPDATE


## Credit to the Source Content
This python notebook uses the content of **Hands-on Lab : INSERT, UPDATE, DELETE** by IBM Skills Network from the [Databases and SQL for Data Science with Python](https://www.coursera.org/learn/sql-data-science) course. The source content uses a different platform, [Datasette](https://github.com/simonw/datasette), which does not utilize Jupyter notebooks.


### Change Log
All versions prior to 2.0, are attributable to IBM Skills Network's version of the **Hands-on Lab : INSERT, UPDATE, DELETE** written for [Datasette](https://github.com/simonw/datasette).


| Date (YYYY-MM-DD) | Version | Changed By    | Change Description        |
| ----------------- | ------- | ------------- | ------------------------- |
|2024-05-xy|2.0|Martin Borja|Ported to Jupyter/Python Notebooks|
|2023-07-11| 1.6| Lakshmi Holla| Updated labs|
|2023-05-11| 1.6| Eric Hao & Vladislav Boyko| Updated Page Frames|
|2023-05-10| 1.5| Eric Hao & Vladislav Boyko| Updated Page Frames|
|2023-05-05| 1.4| Benny Li| Republished|
|2022-08-03| 1.3| Sathya Priya| updated HTML tag|
|2022-07-27| 1.2| Lakshmi Holla| updated HTML tag|
|2020-12-23| 1.1| Steve Ryan| ID Review|
|2020-11-30| 1.0| Sandip Saha Joy| Initial version created|

<hr>

### <h4 align="center"> **Hands-on Lab : INSERT, UPDATE, DELETE** © IBM Corporation 2023. All rights reserved. <h4/>
### <h4 align="center"> **INSERT, UPDATE, DELETE in SQL using Jupyter Notebooks** © Martin John Hilario Borja 2024. All rights reserved. <h4/>

<p>